# Analisis Data Harga Bahan Pokok Bulanan Tahun 2019 - 2023 Kab. Banyumas

Disusun oleh, <br>
Nama        : Chiquito Shaduq Aurick Fulvian <br>
NIM         : 1301210284 <br>
Jurusan     : S1 Informatika <br>
Universitas : Universitas Telkom Bandung

Tujuan dari analisis ini adalah untuk menemukan pola, tren, dan wawasan yang terdapat dari data harga bahan pokok tahun 2019 - 2023 Kabupaten Banyumas. <br>
<br>
Data bersumber dari Dinas Perindustrian dan Perdagangan Kabupaten Banyumas langsung dengan tambahan data dari https://sigaokmas.banyumaskab.go.id dan https://www.bi.go.id/hargapangan

In [1]:
# load packages
import openpyxl as xl
from datetime import datetime
import pandas as pd
import numpy as np
import plotly.express as px

## 1. Data Loading

In [2]:
# load data
wb = xl.load_workbook('data/data_harga/data_bahan_pokok.xlsx')
print(wb.sheetnames)

['2019', '2020', '2021', '2022', '2023']


In [3]:
# check if the data have already loaded
ws = wb['2019'] 
ws.cell(row = 52, column = 2).value == None


True

In [4]:
# create a list for data name
list_bahan = []
for sheet_name in wb.sheetnames:
    init_row = 7
    ws = wb[sheet_name]
    curr_value = ws.cell(row = init_row, column = 2).value
    while curr_value != None :
        if curr_value not in list_bahan:
            list_bahan.append(curr_value)
        init_row += 1
        curr_value = ws.cell(row = init_row, column = 2).value

list_bahan

['BERAS',
 '- BERAS IR64 (kw premium)',
 '- BERAS IR64 (kw medium)',
 'GULA PASIR - kristal putih',
 'MINYAK GORENG',
 ' - Minyak Goreng Curah, Tanpa Merek',
 ' - Minyak Goreng Kemasan, Plastik, Bimoli Kl',
 ' - Minyak Goreng Kemasan Sederhana (1 lt)',
 'DAGING SAPI',
 ' - Daging Sapi Lokal, Paha depan (chuck) ',
 ' - Daging Sapi Lokal, Paha Belakang',
 ' - Daging Sapi Lokal, Has luar (Sirloin)',
 ' - Daging Sapi Lokal, Sandung lamur (Brisket',
 ' -Daging Sapi Lokal Tetelan',
 ' - Daging Sapi Impor, Beku',
 ' - Daging Sapi Lokal, Has Dalam (Tenderloin)',
 'DAGING AYAM',
 ' - Daging Ayam Broiler/Ras, utuh',
 ' - Daging Ayam Kampung',
 'TELUR ',
 '- Telur Ayam Negeri',
 '- Telur Ayam Kampung (per 21 biji)',
 'TEPUNG TERIGU (BOGASARI)',
 '- Tepung Terigu Protein Tinggi',
 '-Tepung Terigu Protein Sedang',
 '-Tepung Terigu Protein Rendah',
 'KEDELAI',
 '- Kedelai Kuning Lokal',
 '- Kedelai Ex. Impor',
 'CABAI',
 ' - Cabai, Merah, Besar',
 ' - Cabai, Merah, Keriting',
 ' - Cabai, Rawit, Mera

In [5]:
# create dictionary for data from list_bahan
i = 0
dict_bahan = {}
while i < len(list_bahan):
    if '-' not in list_bahan[i][0:3]:
        dict_bahan[list_bahan[i]] = []
        j = 1
        while i+j < len(list_bahan) and '-' in list_bahan[i+j][0:3]:
           dict_bahan[list_bahan[i]].append(list_bahan[i+j]) 
           j += 1
        if j == 1:
            dict_bahan[list_bahan[i]].append(list_bahan[i])
    i += j
dict_bahan


{'BERAS': ['- BERAS IR64 (kw premium)', '- BERAS IR64 (kw medium)'],
 'GULA PASIR - kristal putih': ['GULA PASIR - kristal putih'],
 'MINYAK GORENG': [' - Minyak Goreng Curah, Tanpa Merek',
  ' - Minyak Goreng Kemasan, Plastik, Bimoli Kl',
  ' - Minyak Goreng Kemasan Sederhana (1 lt)'],
 'DAGING SAPI': [' - Daging Sapi Lokal, Paha depan (chuck) ',
  ' - Daging Sapi Lokal, Paha Belakang',
  ' - Daging Sapi Lokal, Has luar (Sirloin)',
  ' - Daging Sapi Lokal, Sandung lamur (Brisket',
  ' -Daging Sapi Lokal Tetelan',
  ' - Daging Sapi Impor, Beku',
  ' - Daging Sapi Lokal, Has Dalam (Tenderloin)'],
 'DAGING AYAM': [' - Daging Ayam Broiler/Ras, utuh', ' - Daging Ayam Kampung'],
 'TELUR ': ['- Telur Ayam Negeri', '- Telur Ayam Kampung (per 21 biji)'],
 'TEPUNG TERIGU (BOGASARI)': ['- Tepung Terigu Protein Tinggi',
  '-Tepung Terigu Protein Sedang',
  '-Tepung Terigu Protein Rendah'],
 'KEDELAI': ['- Kedelai Kuning Lokal', '- Kedelai Ex. Impor'],
 'CABAI': [' - Cabai, Merah, Besar',
  ' - Ca

In [6]:
dict_bahan['GULA KELAPA'] = ['GULA KELAPA']
dict_bahan['- jagung'] = ['- jagung']
dict_bahan

{'BERAS': ['- BERAS IR64 (kw premium)', '- BERAS IR64 (kw medium)'],
 'GULA PASIR - kristal putih': ['GULA PASIR - kristal putih'],
 'MINYAK GORENG': [' - Minyak Goreng Curah, Tanpa Merek',
  ' - Minyak Goreng Kemasan, Plastik, Bimoli Kl',
  ' - Minyak Goreng Kemasan Sederhana (1 lt)'],
 'DAGING SAPI': [' - Daging Sapi Lokal, Paha depan (chuck) ',
  ' - Daging Sapi Lokal, Paha Belakang',
  ' - Daging Sapi Lokal, Has luar (Sirloin)',
  ' - Daging Sapi Lokal, Sandung lamur (Brisket',
  ' -Daging Sapi Lokal Tetelan',
  ' - Daging Sapi Impor, Beku',
  ' - Daging Sapi Lokal, Has Dalam (Tenderloin)'],
 'DAGING AYAM': [' - Daging Ayam Broiler/Ras, utuh', ' - Daging Ayam Kampung'],
 'TELUR ': ['- Telur Ayam Negeri', '- Telur Ayam Kampung (per 21 biji)'],
 'TEPUNG TERIGU (BOGASARI)': ['- Tepung Terigu Protein Tinggi',
  '-Tepung Terigu Protein Sedang',
  '-Tepung Terigu Protein Rendah'],
 'KEDELAI': ['- Kedelai Kuning Lokal', '- Kedelai Ex. Impor'],
 'CABAI': [' - Cabai, Merah, Besar',
  ' - Ca

In [7]:
# check the total data
total = 0
for i in dict_bahan:
    total += len(dict_bahan[i])
    
total

39

In [8]:
dict_bahan['BERAS']

['- BERAS IR64 (kw premium)', '- BERAS IR64 (kw medium)']

In [9]:
# create a dictionary for dataframe's data
data = {'tanggal': [],
        'nama' : [],
        'satuan': [],
        'harga': [],
        'kategori': []}
for sheet_name in wb.sheetnames:
        curr_row = 7 
        ws = wb[sheet_name]
        kategori = ws.cell(row = curr_row, column = 2).value
        while kategori in dict_bahan:
                if ws.cell(row = curr_row+1, column = 2).value in dict_bahan[kategori]:
                        curr_row += 1
                nama = ws.cell(row = curr_row, column = 2).value
                while nama in dict_bahan[kategori]:
                        satuan = ws.cell(row = curr_row, column = 3).value
                        for i in range(1, 13):
                                data['tanggal'].append(datetime(int(sheet_name), i, 1))
                                data['nama'].append(nama)
                                data['satuan'].append(satuan)
                                data['harga'].append(ws.cell(row = curr_row, column = 3+i).value)
                                data['kategori'].append(kategori)
                        curr_row += 1
                        nama = ws.cell(row = curr_row, column = 2).value
                kategori = ws.cell(row = curr_row, column = 2).value

data         
        

{'tanggal': [datetime.datetime(2019, 1, 1, 0, 0),
  datetime.datetime(2019, 2, 1, 0, 0),
  datetime.datetime(2019, 3, 1, 0, 0),
  datetime.datetime(2019, 4, 1, 0, 0),
  datetime.datetime(2019, 5, 1, 0, 0),
  datetime.datetime(2019, 6, 1, 0, 0),
  datetime.datetime(2019, 7, 1, 0, 0),
  datetime.datetime(2019, 8, 1, 0, 0),
  datetime.datetime(2019, 9, 1, 0, 0),
  datetime.datetime(2019, 10, 1, 0, 0),
  datetime.datetime(2019, 11, 1, 0, 0),
  datetime.datetime(2019, 12, 1, 0, 0),
  datetime.datetime(2019, 1, 1, 0, 0),
  datetime.datetime(2019, 2, 1, 0, 0),
  datetime.datetime(2019, 3, 1, 0, 0),
  datetime.datetime(2019, 4, 1, 0, 0),
  datetime.datetime(2019, 5, 1, 0, 0),
  datetime.datetime(2019, 6, 1, 0, 0),
  datetime.datetime(2019, 7, 1, 0, 0),
  datetime.datetime(2019, 8, 1, 0, 0),
  datetime.datetime(2019, 9, 1, 0, 0),
  datetime.datetime(2019, 10, 1, 0, 0),
  datetime.datetime(2019, 11, 1, 0, 0),
  datetime.datetime(2019, 12, 1, 0, 0),
  datetime.datetime(2019, 1, 1, 0, 0),
  dateti

In [10]:
# create dataframe
df = pd.DataFrame(data)
df

,tanggal,nama,satuan,harga,kategori
0,2019-01-01,- BERAS IR64 (kw premium),kg,13000.0,BERAS
1,2019-02-01,- BERAS IR64 (kw premium),kg,13000.0,BERAS
2,2019-03-01,- BERAS IR64 (kw premium),kg,13000.0,BERAS
3,2019-04-01,- BERAS IR64 (kw premium),kg,13000.0,BERAS
4,2019-05-01,- BERAS IR64 (kw premium),kg,13000.0,BERAS
...,...,...,...,...,...
2227,2023-08-01,GULA KELAPA,kg,17000.0,GULA KELAPA
2228,2023-09-01,GULA KELAPA,kg,17000.0,GULA KELAPA
2229,2023-10-01,GULA KELAPA,kg,17000.0,GULA KELAPA
2230,2023-11-01,GULA KELAPA,kg,17000.0,GULA KELAPA


## 2. Data Pre-processing

In [11]:
# lower string in nama and kategori
df['nama'] = df['nama'].str.lower()
df['kategori'] = df['kategori'].str.lower()
df

,tanggal,nama,satuan,harga,kategori
0,2019-01-01,- beras ir64 (kw premium),kg,13000.0,beras
1,2019-02-01,- beras ir64 (kw premium),kg,13000.0,beras
2,2019-03-01,- beras ir64 (kw premium),kg,13000.0,beras
3,2019-04-01,- beras ir64 (kw premium),kg,13000.0,beras
4,2019-05-01,- beras ir64 (kw premium),kg,13000.0,beras
...,...,...,...,...,...
2227,2023-08-01,gula kelapa,kg,17000.0,gula kelapa
2228,2023-09-01,gula kelapa,kg,17000.0,gula kelapa
2229,2023-10-01,gula kelapa,kg,17000.0,gula kelapa
2230,2023-11-01,gula kelapa,kg,17000.0,gula kelapa


In [12]:
# remove '-' in nama and kategori
df['nama'] = df['nama'].str.replace('-', '')
df['kategori'] = df['kategori'].str.replace('-', '')
df

,tanggal,nama,satuan,harga,kategori
0,2019-01-01,beras ir64 (kw premium),kg,13000.0,beras
1,2019-02-01,beras ir64 (kw premium),kg,13000.0,beras
2,2019-03-01,beras ir64 (kw premium),kg,13000.0,beras
3,2019-04-01,beras ir64 (kw premium),kg,13000.0,beras
4,2019-05-01,beras ir64 (kw premium),kg,13000.0,beras
...,...,...,...,...,...
2227,2023-08-01,gula kelapa,kg,17000.0,gula kelapa
2228,2023-09-01,gula kelapa,kg,17000.0,gula kelapa
2229,2023-10-01,gula kelapa,kg,17000.0,gula kelapa
2230,2023-11-01,gula kelapa,kg,17000.0,gula kelapa


In [13]:
# remove whitespace in nama and kategori
df['nama'] = df['nama'].str.strip()
df['kategori'] = df['kategori'].str.strip()
df

,tanggal,nama,satuan,harga,kategori
0,2019-01-01,beras ir64 (kw premium),kg,13000.0,beras
1,2019-02-01,beras ir64 (kw premium),kg,13000.0,beras
2,2019-03-01,beras ir64 (kw premium),kg,13000.0,beras
3,2019-04-01,beras ir64 (kw premium),kg,13000.0,beras
4,2019-05-01,beras ir64 (kw premium),kg,13000.0,beras
...,...,...,...,...,...
2227,2023-08-01,gula kelapa,kg,17000.0,gula kelapa
2228,2023-09-01,gula kelapa,kg,17000.0,gula kelapa
2229,2023-10-01,gula kelapa,kg,17000.0,gula kelapa
2230,2023-11-01,gula kelapa,kg,17000.0,gula kelapa


In [14]:
# check all kategori values
df['kategori'].unique()


array(['beras', 'gula pasir  kristal putih', 'minyak goreng',
       'daging sapi', 'daging ayam', 'telur', 'tepung terigu (bogasari)',
       'kedelai', 'cabai', 'bawang merah  lokal', 'bawang putih',
       'ikan laut teri', 'garam beryodium  halus',
       'mie instantindomie ayam bawang', 'kacang tanah', 'kacang hijau',
       'ketela pohon', 'gula kelapa', 'jagung', 'ikan laut kembung',
       'susu dancow'], dtype=object)

In [15]:
# check all nama values
df['nama'].unique()

array(['beras ir64 (kw premium)', 'beras ir64 (kw medium)',
       'gula pasir  kristal putih', 'minyak goreng curah, tanpa merek',
       'minyak goreng kemasan, plastik, bimoli kl',
       'minyak goreng kemasan sederhana (1 lt)',
       'daging sapi lokal, paha depan (chuck)',
       'daging sapi lokal, paha belakang',
       'daging sapi lokal, has luar (sirloin)',
       'daging sapi lokal, sandung lamur (brisket',
       'daging sapi lokal tetelan', 'daging sapi impor, beku',
       'daging sapi lokal, has dalam (tenderloin)',
       'daging ayam broiler/ras, utuh', 'daging ayam kampung',
       'telur ayam negeri', 'telur ayam kampung (per 21 biji)',
       'tepung terigu protein tinggi', 'tepung terigu protein sedang',
       'tepung terigu protein rendah', 'kedelai kuning lokal',
       'kedelai ex. impor', 'cabai, merah, besar',
       'cabai, merah, keriting', 'cabai, rawit, merah',
       'cabai rawit hijau', 'bawang merah  lokal',
       'bawang putih impor, honan', 'bawan

In [16]:
# fix kategori
df_processed = df.copy()
df_processed.loc[df_processed['kategori'] == 'gula kelapa', 'kategori'] = 'gula'
df_processed.loc[df_processed['kategori'] == 'gula pasir  kristal putih', 'kategori'] = 'gula'
df_processed.loc[df_processed['kategori'] == 'bawang putih', 'kategori'] = 'bawang'
df_processed.loc[df_processed['kategori'] == 'bawang merah  lokal', 'kategori'] = 'bawang'
df_processed.loc[df_processed['kategori'] == 'ikan laut teri', 'kategori'] = 'ikan laut'
df_processed.loc[df_processed['kategori'] == 'ikan laut kembung', 'kategori'] = 'ikan laut'
df_processed.loc[df_processed['kategori'] == 'garam beryodium  halus', 'kategori'] = 'garam'
df_processed.loc[df_processed['kategori'] == 'mie instantindomie ayam bawang', 'kategori'] = 'mie instant'
df_processed.loc[df_processed['kategori'] == 'kacang tanah' , 'kategori'] = 'kacang'
df_processed.loc[df_processed['kategori'] == 'kacang hijau', 'kategori'] = 'kacang'
df_processed.loc[df_processed['kategori'] == 'mie instant', 'nama'] = 'indomie ayam bawang'
df_processed['kategori'].unique()

array(['beras', 'gula', 'minyak goreng', 'daging sapi', 'daging ayam',
       'telur', 'tepung terigu (bogasari)', 'kedelai', 'cabai', 'bawang',
       'ikan laut', 'garam', 'mie instant', 'kacang', 'ketela pohon',
       'jagung', 'susu dancow'], dtype=object)

In [17]:
df_processed['nama'].unique()

array(['beras ir64 (kw premium)', 'beras ir64 (kw medium)',
       'gula pasir  kristal putih', 'minyak goreng curah, tanpa merek',
       'minyak goreng kemasan, plastik, bimoli kl',
       'minyak goreng kemasan sederhana (1 lt)',
       'daging sapi lokal, paha depan (chuck)',
       'daging sapi lokal, paha belakang',
       'daging sapi lokal, has luar (sirloin)',
       'daging sapi lokal, sandung lamur (brisket',
       'daging sapi lokal tetelan', 'daging sapi impor, beku',
       'daging sapi lokal, has dalam (tenderloin)',
       'daging ayam broiler/ras, utuh', 'daging ayam kampung',
       'telur ayam negeri', 'telur ayam kampung (per 21 biji)',
       'tepung terigu protein tinggi', 'tepung terigu protein sedang',
       'tepung terigu protein rendah', 'kedelai kuning lokal',
       'kedelai ex. impor', 'cabai, merah, besar',
       'cabai, merah, keriting', 'cabai, rawit, merah',
       'cabai rawit hijau', 'bawang merah  lokal',
       'bawang putih impor, honan', 'bawan

In [18]:
# fix nama
df_processed.loc[df_processed['nama'] == 'bawang merah  lokal', 'nama'] = 'bawang merah lokal'
df_processed.loc[df_processed['nama'] == 'cabai, merah, besar', 'nama'] = 'cabai merah besar'
df_processed.loc[df_processed['nama'] == 'cabai, merah, keriting', 'nama'] = 'cabai merah keriting'
df_processed.loc[df_processed['nama'] == 'cabai, rawit, merah', 'nama'] = 'cabai rawit merah'
df_processed.loc[df_processed['nama'] == 'garam beryodium  halus', 'nama'] = 'garam beryodium halus'
df_processed.loc[df_processed['nama'] == 'gula pasir  kristal putih', 'nama'] = 'gula pasir kristal putih'
df_processed['nama'].unique()

array(['beras ir64 (kw premium)', 'beras ir64 (kw medium)',
       'gula pasir kristal putih', 'minyak goreng curah, tanpa merek',
       'minyak goreng kemasan, plastik, bimoli kl',
       'minyak goreng kemasan sederhana (1 lt)',
       'daging sapi lokal, paha depan (chuck)',
       'daging sapi lokal, paha belakang',
       'daging sapi lokal, has luar (sirloin)',
       'daging sapi lokal, sandung lamur (brisket',
       'daging sapi lokal tetelan', 'daging sapi impor, beku',
       'daging sapi lokal, has dalam (tenderloin)',
       'daging ayam broiler/ras, utuh', 'daging ayam kampung',
       'telur ayam negeri', 'telur ayam kampung (per 21 biji)',
       'tepung terigu protein tinggi', 'tepung terigu protein sedang',
       'tepung terigu protein rendah', 'kedelai kuning lokal',
       'kedelai ex. impor', 'cabai merah besar', 'cabai merah keriting',
       'cabai rawit merah', 'cabai rawit hijau', 'bawang merah lokal',
       'bawang putih impor, honan', 'bawang putih impor, 

In [19]:
df = df_processed.copy()

In [20]:
# check duplicate
df.duplicated().sum()

np.int64(0)

In [21]:
#check missing value
df.isna().sum()

tanggal      0
nama         0
satuan      24
harga       24
kategori     0
dtype: int64

In [22]:
df.loc[df['satuan'].isna(), 'nama']

1668    ikan laut kembung
1669    ikan laut kembung
1670    ikan laut kembung
1671    ikan laut kembung
1672    ikan laut kembung
1673    ikan laut kembung
1674    ikan laut kembung
1675    ikan laut kembung
1676    ikan laut kembung
1677    ikan laut kembung
1678    ikan laut kembung
1679    ikan laut kembung
2136    ikan laut kembung
2137    ikan laut kembung
2138    ikan laut kembung
2139    ikan laut kembung
2140    ikan laut kembung
2141    ikan laut kembung
2142    ikan laut kembung
2143    ikan laut kembung
2144    ikan laut kembung
2145    ikan laut kembung
2146    ikan laut kembung
2147    ikan laut kembung
Name: nama, dtype: object

In [23]:
# fill missing values
df['satuan'] = df['satuan'].fillna('kg')


In [24]:
df.describe()

,tanggal,harga
count,2232,2208.000000
mean,2021-07-03 22:27:05.806451712,41861.489023
min,2019-01-01 00:00:00,2100.000000
25%,2020-04-01 00:00:00,12442.857143
50%,2021-07-16 12:00:00,24000.000000
75%,2022-10-01 00:00:00,57309.090909
max,2023-12-01 00:00:00,146875.000000
std,NaN,41076.850220


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2232 entries, 0 to 2231
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   tanggal   2232 non-null   datetime64[ns]
 1   nama      2232 non-null   object        
 2   satuan    2232 non-null   object        
 3   harga     2208 non-null   float64       
 4   kategori  2232 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 87.3+ KB


In [26]:
df.isna().sum()

tanggal      0
nama         0
satuan       0
harga       24
kategori     0
dtype: int64

In [27]:
df.loc[df['harga'].isna()]

,tanggal,nama,satuan,harga,kategori
132,2019-01-01,"daging sapi impor, beku",kg,NaN,daging sapi
133,2019-02-01,"daging sapi impor, beku",kg,NaN,daging sapi
134,2019-03-01,"daging sapi impor, beku",kg,NaN,daging sapi
135,2019-04-01,"daging sapi impor, beku",kg,NaN,daging sapi
136,2019-05-01,"daging sapi impor, beku",kg,NaN,daging sapi
137,2019-06-01,"daging sapi impor, beku",kg,NaN,daging sapi
138,2019-07-01,"daging sapi impor, beku",kg,NaN,daging sapi
139,2019-08-01,"daging sapi impor, beku",kg,NaN,daging sapi
140,2019-09-01,"daging sapi impor, beku",kg,NaN,daging sapi
141,2019-10-01,"daging sapi impor, beku",kg,NaN,daging sapi


In [28]:
df['harga'] = df['harga'].fillna(np.nan)

In [29]:
df.isna().sum()

tanggal      0
nama         0
satuan       0
harga       24
kategori     0
dtype: int64

In [30]:
# export dataframe into csv file
df.to_csv('data/dataframe_data.csv')

## 3. EDA

In [31]:
df = pd.read_csv('data/dataframe_data.csv', index_col = 0)
df

,tanggal,nama,satuan,harga,kategori
0,2019-01-01,beras ir64 (kw premium),kg,13000.0,beras
1,2019-02-01,beras ir64 (kw premium),kg,13000.0,beras
2,2019-03-01,beras ir64 (kw premium),kg,13000.0,beras
3,2019-04-01,beras ir64 (kw premium),kg,13000.0,beras
4,2019-05-01,beras ir64 (kw premium),kg,13000.0,beras
...,...,...,...,...,...
2227,2023-08-01,gula kelapa,kg,17000.0,gula
2228,2023-09-01,gula kelapa,kg,17000.0,gula
2229,2023-10-01,gula kelapa,kg,17000.0,gula
2230,2023-11-01,gula kelapa,kg,17000.0,gula


In [32]:
df.describe(include='all')

,tanggal,nama,satuan,harga,kategori
count,2232,2232,2232,2208.000000,2232
unique,60,39,3,NaN,17
top,2023-01-01,beras ir64 (kw premium),kg,NaN,daging sapi
freq,39,60,1992,NaN,420
mean,NaN,NaN,NaN,41861.489023,NaN
std,NaN,NaN,NaN,41076.850220,NaN
min,NaN,NaN,NaN,2100.000000,NaN
25%,NaN,NaN,NaN,12442.857143,NaN
50%,NaN,NaN,NaN,24000.000000,NaN
75%,NaN,NaN,NaN,57309.090909,NaN


In [33]:
df.loc[df['harga'] == df['harga'].max()]

,tanggal,nama,satuan,harga,kategori
1839,2023-04-01,"daging sapi lokal, paha depan (chuck)",kg,146875.0,daging sapi
1851,2023-04-01,"daging sapi lokal, paha belakang",kg,146875.0,daging sapi
1863,2023-04-01,"daging sapi lokal, has luar (sirloin)",kg,146875.0,daging sapi
1911,2023-04-01,"daging sapi lokal, has dalam (tenderloin)",kg,146875.0,daging sapi


In [34]:
df.loc[df['harga'] == df['harga'].min()]

,tanggal,nama,satuan,harga,kategori
372,2019-01-01,indomie ayam bawang,bungkus,2100.0,mie instant
373,2019-02-01,indomie ayam bawang,bungkus,2100.0,mie instant
374,2019-03-01,indomie ayam bawang,bungkus,2100.0,mie instant
375,2019-04-01,indomie ayam bawang,bungkus,2100.0,mie instant
376,2019-05-01,indomie ayam bawang,bungkus,2100.0,mie instant
377,2019-06-01,indomie ayam bawang,bungkus,2100.0,mie instant
378,2019-07-01,indomie ayam bawang,bungkus,2100.0,mie instant
379,2019-08-01,indomie ayam bawang,bungkus,2100.0,mie instant
380,2019-09-01,indomie ayam bawang,bungkus,2100.0,mie instant
381,2019-10-01,indomie ayam bawang,bungkus,2100.0,mie instant


In [35]:
max_prices = df.groupby('nama')['harga'].max().reset_index()
max_prices.sort_values('harga', ascending=False)[:5].reset_index(drop=True)

,nama,harga
0,"daging sapi lokal, paha depan (chuck)",146875.0
1,"daging sapi lokal, has luar (sirloin)",146875.0
2,"daging sapi lokal, has dalam (tenderloin)",146875.0
3,"daging sapi lokal, paha belakang",146875.0
4,daging sapi lokal tetelan,128125.0


In [36]:
min_prices = df.groupby('nama')['harga'].min().reset_index()
min_prices.sort_values('harga', ascending=True)[:5].reset_index(drop=True)

,nama,harga
0,indomie ayam bawang,2100.000000
1,ketela pohon,2666.666667
2,kedelai ex. impor,6705.263158
3,jagung,7000.000000
4,garam beryodium halus,8000.000000


In [37]:
# Create a Plotly bar chart to visualize the highest prices for each commodity
fig = px.bar(max_prices, x='nama', y='harga', 
             title='Harga Tertinggi untuk Bahan Pokok',
             labels={'nama': 'Bahan Pokok', 'harga': 'Harga Tertinggi'},
             template='plotly')

# Show the plot
fig.show()

In [50]:
std_prices = df.groupby('nama')['harga'].std().reset_index()
std_prices.columns = ['nama', 'std_dev']
std_prices.sort_values(by = 'std_dev',ascending=False, inplace=True, ignore_index=True)
std_prices[:5]

,nama,std_dev
0,cabai rawit merah,20279.057143
1,cabai merah keriting,15513.338253
2,cabai merah besar,14775.015859
3,cabai rawit hijau,12689.729861
4,"daging sapi lokal, has dalam (tenderloin)",8786.773370
